# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [52]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

### Definitions

In [2]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_SINGLE_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_single_step_pos_fr"
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 135


### helper functions

In [15]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [3]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,pos,age_pos,test,age_test,day_num,nb_cases,train
date,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,882,60.987528,38613,55.469013,3,141109,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,981,60.434251,41396,54.810127,4,142090,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,1031,59.838991,46836,54.322679,5,143121,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,291,60.158076,16094,54.354356,6,143412,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,139,61.568345,6245,58.054604,0,143551,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-22,288.068333,2020-10-22,293.946667,66.254237,90.338983,48940,47.908214,253252,46.256278,4,1099014,False
2020-10-23,287.036441,2020-10-23,292.078814,70.017241,92.896552,52533,48.007462,277455,46.673734,5,1151547,False
2020-10-24,286.104237,2020-10-24,292.195763,66.327586,91.137931,23577,47.540993,120734,46.430161,6,1175124,False


### prepare features

In [4]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,882,38613,3,60.987528,55.469013
2020-05-14,285.050000,290.963333,59.406780,84.847458,981,41396,4,60.434251,54.810127
2020-05-15,285.308333,291.920000,57.372881,82.966102,1031,46836,5,59.838991,54.322679
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16094,6,60.158076,54.354356
2020-05-17,285.598333,294.446667,49.879310,85.500000,139,6245,0,61.568345,58.054604
...,...,...,...,...,...,...,...,...,...
2020-10-22,288.068333,293.946667,66.254237,90.338983,48940,253252,4,47.908214,46.256278
2020-10-23,287.036441,292.078814,70.017241,92.896552,52533,277455,5,48.007462,46.673734
2020-10-24,286.104237,292.195763,66.327586,91.137931,23577,120734,6,47.540993,46.430161


### Prepare dataset

In [5]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [6]:
dataset.shape[1]

9

In [7]:
PAST_HISTORY

14

In [8]:
dataset.shape[1]

9

### Load model

In [9]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x63c094f90> and <tensorflow.python.keras.layers.core.Dropout object at 0x139c9bbd0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63c10f490> and <tensorflow.python.keras.layers.core.Dense object at 0x63c119a50>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63c10f490> and <tensorflow.python.keras.layers.core.Dropout object at 0x139c9bbd0>).
CPU times: user 2.45 s, sys: 173 ms, total: 2.62 s
Wall time: 3.02 s


In [10]:
multi_step_model.inputs[0].dtype

tf.float32

In [11]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_lstm/assets


### Save model TFlite

In [16]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20201030_17_33_58.tflite moved!


In [17]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

6104

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [18]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]
9


array([[  814.1582 ,   583.9282 ,   606.40186,   814.5974 ,  1025.219  ,
         1217.3073 ,  1303.9923 ,  1047.2645 ,   749.4304 ,  1179.5789 ,
         1440.1638 ,  1488.9644 ,  1699.5714 ,  1812.4594 ,  1557.5139 ,
         1246.2732 ,  1179.1752 ,  1793.6415 ,  2323.6118 ,  2527.147  ,
         2529.7927 ,  2188.1914 ,  1579.2222 ,  2183.4146 ,  3467.8218 ,
         3882.2388 ,  3890.2764 ,  4097.167  ,  3720.858  ,  2534.9634 ,
         2686.615  ,  4838.0923 ,  5727.835  ,  5415.9976 ,  5833.218  ,
         5480.083  ,  2745.1045 ,  2892.1353 ,  6756.036  ,  8333.597  ,
         7592.1436 ,  8443.741  ,  7841.1523 ,  3621.1506 ,  3887.908  ,
        10062.342  , 11882.219  ,  9718.192  , 11525.329  ,  8651.092  ,
         3160.4739 ,  4677.614  , 11304.128  , 12989.707  , 10908.019  ,
        12652.731  ,  8159.621  ,  4238.619  ,  5153.3184 , 10712.284  ,
        11636.399  ,  9384.854  , 11557.597  ]], dtype=float32)

#### Future days

In [19]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [20]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [21]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [22]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [23]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [24]:
x_multi.shape

(1, 14, 9)

In [25]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [26]:
dataset.shape

(167, 9)

In [27]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]


In [28]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [29]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [30]:
y_multi_pred

array([[-5.1732117e-01, -5.7452375e-01, -5.6893998e-01, -5.1721203e-01,
        -4.6488130e-01, -4.1715536e-01, -3.9561772e-01, -4.5940390e-01,
        -5.3340334e-01, -4.2652929e-01, -3.6178476e-01, -3.4965986e-01,
        -2.9733273e-01, -2.6928478e-01, -3.3262813e-01, -4.0995851e-01,
        -4.2662960e-01, -2.7396023e-01, -1.4228460e-01, -9.1714621e-02,
        -9.1057241e-02, -1.7593101e-01, -3.2723454e-01, -1.7711782e-01,
         1.4200398e-01,  2.4496943e-01,  2.4696645e-01,  2.9837018e-01,
         2.0487297e-01, -8.9772537e-02, -5.2093413e-02,  4.8245931e-01,
         7.0352340e-01,  6.2604475e-01,  7.2970659e-01,  6.4196730e-01,
        -3.7561230e-02, -1.0301718e-03,  9.5898873e-01,  1.3509469e+00,
         1.1667266e+00,  1.3783133e+00,  1.2285950e+00,  1.8009989e-01,
         2.4637797e-01,  1.7804682e+00,  2.2326322e+00,  1.6949615e+00,
         2.1439600e+00,  1.4298314e+00,  6.5640852e-02,  4.4258708e-01,
         2.0890007e+00,  2.5077970e+00,  1.9905839e+00,  2.42407

In [31]:
y_multi_pred_out

array([[-5.17321110e-01, -5.74523807e-01, -5.68940043e-01,
        -5.17212033e-01, -4.64881212e-01, -4.17155355e-01,
        -3.95617664e-01, -4.59403932e-01, -5.33403337e-01,
        -4.26529258e-01, -3.61784786e-01, -3.49659920e-01,
        -2.97332764e-01, -2.69284815e-01, -3.32628131e-01,
        -4.09958512e-01, -4.26629603e-01, -2.73960263e-01,
        -1.42284647e-01, -9.17146355e-02, -9.10572708e-02,
        -1.75930962e-01, -3.27234447e-01, -1.77117839e-01,
         1.42003983e-01,  2.44969487e-01,  2.46966541e-01,
         2.98370153e-01,  2.04872906e-01, -8.97726566e-02,
        -5.20933904e-02,  4.82459396e-01,  7.03523457e-01,
         6.26044750e-01,  7.29706585e-01,  6.41967118e-01,
        -3.75612527e-02, -1.03015546e-03,  9.58988726e-01,
         1.35094690e+00,  1.16672659e+00,  1.37831330e+00,
         1.22859478e+00,  1.80100009e-01,  2.46377558e-01,
         1.78046775e+00,  2.23263192e+00,  1.69496131e+00,
         2.14395952e+00,  1.42983115e+00,  6.56409860e-0

In [32]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [33]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [34]:
y_future_pred_out

array([[2.07842183, 2.39811206, 6.6593318 , 5.35568762, 2.04537964,
        4.28097916, 7.50129271]])

In [35]:
y_future_pred

array([[2.0784218, 2.398112 , 6.659332 , 5.3556876, 2.0453794, 4.2809796,
        7.501292 ]], dtype=float32)

In [36]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [37]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [38]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [39]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [40]:
!./deploy_serverless.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Injecting required Python packages to package...
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service tensorflow-lite-on-aws-lambda.zip file to S3 (18.4 MB)...
Serverless: Validating template...
Serverless: Updating Stack...
Serverless: Checking Stack update progress...
CloudF

### API AWS real Test

#### Past days

In [41]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[58 - 72]
[65 - 79]
[72 - 86]
[79 - 93]
[86 - 100]
[93 - 107]
[100 - 114]
[107 - 121]
[114 - 128]
status code :  200
[[[-0.5173211097717285, -0.5745238065719604, -0.5689399838447571, -0.5172120332717896, -0.4648812413215637, -0.417155385017395, -0.39561766386032104]], [[-0.4594038724899292, -0.5334033370018005, -0.42652931809425354, -0.36178475618362427, -0.34965986013412476, -0.2973327338695526, -0.26928478479385376]], [[-0.3326281011104584, -0.40995851159095764, -0.42662960290908813, -0.2739602327346802, -0.14228461682796478, -0.09171462059020996, -0.09105727076530457]], [[-0.1759309470653534, -0.3272344470024109, -0.17711782455444336, 0.14200395345687866, 0.2449694275856018, 0.24696645140647888, 0.2983701229095459]], [[0.2048729956150055, -0.08977252244949341, -0.052093494683504105, 0.48245933651924133, 0.7035234570503235, 0.626044750213623, 0.7297065854072571]], [[0.6419671773910522, -0.037561241537332535, -0.001030157320201397, 0.9589887857437134, 1.3509469032287598, 1.16672646999

In [42]:
len(json_list_list_x)

23764

In [43]:
resp.json()

[[[-0.5173211097717285,
   -0.5745238065719604,
   -0.5689399838447571,
   -0.5172120332717896,
   -0.4648812413215637,
   -0.417155385017395,
   -0.39561766386032104]],
 [[-0.4594038724899292,
   -0.5334033370018005,
   -0.42652931809425354,
   -0.36178475618362427,
   -0.34965986013412476,
   -0.2973327338695526,
   -0.26928478479385376]],
 [[-0.3326281011104584,
   -0.40995851159095764,
   -0.42662960290908813,
   -0.2739602327346802,
   -0.14228461682796478,
   -0.09171462059020996,
   -0.09105727076530457]],
 [[-0.1759309470653534,
   -0.3272344470024109,
   -0.17711782455444336,
   0.14200395345687866,
   0.2449694275856018,
   0.24696645140647888,
   0.2983701229095459]],
 [[0.2048729956150055,
   -0.08977252244949341,
   -0.052093494683504105,
   0.48245933651924133,
   0.7035234570503235,
   0.626044750213623,
   0.7297065854072571]],
 [[0.6419671773910522,
   -0.037561241537332535,
   -0.001030157320201397,
   0.9589887857437134,
   1.3509469032287598,
   1.1667264699935913,


In [44]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [45]:
y_multi_pred_out

array([[-5.17321110e-01, -5.74523807e-01, -5.68939984e-01,
        -5.17212033e-01, -4.64881241e-01, -4.17155385e-01,
        -3.95617664e-01, -4.59403872e-01, -5.33403337e-01,
        -4.26529318e-01, -3.61784756e-01, -3.49659860e-01,
        -2.97332734e-01, -2.69284785e-01, -3.32628101e-01,
        -4.09958512e-01, -4.26629603e-01, -2.73960233e-01,
        -1.42284617e-01, -9.17146206e-02, -9.10572708e-02,
        -1.75930947e-01, -3.27234447e-01, -1.77117825e-01,
         1.42003953e-01,  2.44969428e-01,  2.46966451e-01,
         2.98370123e-01,  2.04872996e-01, -8.97725224e-02,
        -5.20934947e-02,  4.82459337e-01,  7.03523457e-01,
         6.26044750e-01,  7.29706585e-01,  6.41967177e-01,
        -3.75612415e-02, -1.03015732e-03,  9.58988786e-01,
         1.35094690e+00,  1.16672647e+00,  1.37831330e+00,
         1.22859466e+00,  1.80099696e-01,  2.46377736e-01,
         1.78046775e+00,  2.23263192e+00,  1.69496143e+00,
         2.14395976e+00,  1.42983127e+00,  6.56405985e-0

In [46]:
y_multi_pred

array([[-5.1732117e-01, -5.7452375e-01, -5.6893998e-01, -5.1721203e-01,
        -4.6488130e-01, -4.1715536e-01, -3.9561772e-01, -4.5940390e-01,
        -5.3340334e-01, -4.2652929e-01, -3.6178476e-01, -3.4965986e-01,
        -2.9733273e-01, -2.6928478e-01, -3.3262813e-01, -4.0995851e-01,
        -4.2662960e-01, -2.7396023e-01, -1.4228460e-01, -9.1714621e-02,
        -9.1057241e-02, -1.7593101e-01, -3.2723454e-01, -1.7711782e-01,
         1.4200398e-01,  2.4496943e-01,  2.4696645e-01,  2.9837018e-01,
         2.0487297e-01, -8.9772537e-02, -5.2093413e-02,  4.8245931e-01,
         7.0352340e-01,  6.2604475e-01,  7.2970659e-01,  6.4196730e-01,
        -3.7561230e-02, -1.0301718e-03,  9.5898873e-01,  1.3509469e+00,
         1.1667266e+00,  1.3783133e+00,  1.2285950e+00,  1.8009989e-01,
         2.4637797e-01,  1.7804682e+00,  2.2326322e+00,  1.6949615e+00,
         2.1439600e+00,  1.4298314e+00,  6.5640852e-02,  4.4258708e-01,
         2.0890007e+00,  2.5077970e+00,  1.9905839e+00,  2.42407

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [48]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[2.0784218311309814, 2.3981120586395264, 6.659331321716309, 5.355687618255615, 2.045379400253296, 4.280978679656982, 7.5012922286987305]]]


In [49]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[2.07842183, 2.39811206, 6.65933132, 5.35568762, 2.0453794 ,
        4.28097868, 7.50129223]])

In [50]:
y_future_pred

array([[2.0784218, 2.398112 , 6.659332 , 5.3556876, 2.0453794, 4.2809796,
        7.501292 ]], dtype=float32)

In [51]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
